<a href="https://colab.research.google.com/github/ubsuny/PHY386/blob/Homework2025/2025/HW/DDarrigo-Brick/Hw_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# Install required libraries
!pip install astropy scikit-learn tensorflow matplotlib numpy photutils auto-stretch astroquery



In [77]:
import tensorflow
import matplotlib
import numpy
import photutils
import auto_stretch
import astroquery
import sklearn
import pandas as pd
import requests
from io import BytesIO
from astroquery.simbad import Simbad
import astropy.io.fits as fits
from astropy.wcs import WCS
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u

In [33]:



# Fetch the FITS file from the URL
url = "https://raw.githubusercontent.com/ubsuny/PHY386/main/data/1.fits"  # Use raw content link
response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad status codes

# Open the FITS file from the response content
with fits.open(BytesIO(response.content)) as hdul:
    # 1. Determine WCS coordinates
    wcs = WCS(hdul[0].header)
    print("WCS information:")
    print(wcs)

    # 2. Generate a list of the 10 brightest objects
    data = hdul[0].data
    print("\nData shape:", data.shape)
    # Flatten the array, sort by brightness, select top 10
    flattened_data = data.flatten()  # Flatten to 1D array
    brightest_indices = flattened_data.argsort()[::-1][:10]
    brightest_values = flattened_data[brightest_indices]

# Convert flattened indices back to multi-dimensional indices
multi_dimensional_indices = numpy.unravel_index(brightest_indices, data.shape)

# Access the relevant coordinates based on the dimensions
# Assuming the desired dimensions are the last two
x_coords = multi_dimensional_indices[1]  # Second dimension
y_coords = multi_dimensional_indices[2]  # Third dimension

brightest_coords = list(zip(x_coords, y_coords))

df = pd.DataFrame({'Brightness': brightest_values, 'Coordinates': brightest_coords})
print("\n10 Brightest Objects:")
df


WCS information:
WCS Keywords

Number of WCS axes: 3
CTYPE : 'RA---TAN' 'DEC--TAN' '' 
CRVAL : 79.1719340780092 46.0087592970223 0.0 
CRPIX : 540.0 960.0 0.0 
PC1_1 PC1_2 PC1_3  : 0.999797158835514 -0.0201405358527186 0.0 
PC2_1 PC2_2 PC2_3  : 0.0201622861616532 0.999796720447079 0.0 
PC3_1 PC3_2 PC3_3  : 0.0 0.0 1.0 
CDELT : -0.00110890832843152 0.00110891393928793 1.0 
NAXIS : 1080  1920  3

Data shape: (3, 1920, 1080)

10 Brightest Objects:


,Brightness,Coordinates
0,1.000000,"(947, 539)"
1,0.979089,"(948, 539)"
2,0.978442,"(947, 538)"
3,0.977624,"(947, 540)"
4,0.976830,"(946, 539)"
5,0.959355,"(948, 540)"
6,0.957943,"(946, 538)"
7,0.955842,"(948, 538)"
8,0.952842,"(946, 540)"
9,0.916916,"(949, 539)"


In [88]:

# Customize Simbad query fields
custom_simbad = Simbad()
custom_simbad.TIMEOUT = 120
custom_simbad.add_votable_fields('otype', 'sp', 'distance', 'flux(V)')  # Object type, Spectral Type, Distance, V Magnitude

# Storage for results
results = []

# Query each bright object's sky coordinate
for idx, coord in enumerate(sky_coords):
    try:
        result = custom_simbad.query_region(coord, radius=50 * u.arcsec)

        if result is not None and len(result) > 0:
            entry = result[0]
            name = entry['MAIN_ID'].decode('utf-8') if isinstance(entry['MAIN_ID'], bytes) else entry['MAIN_ID']
            otype = entry['OTYPE'].decode('utf-8') if isinstance(entry['OTYPE'], bytes) else entry['OTYPE']
            spectral_type = entry['SP_TYPE'].decode('utf-8') if isinstance(entry['SP_TYPE'], bytes) else entry['SP_TYPE']
            distance_pc = entry['Distance_distance']  # distance in parsecs
            apparent_mag = entry['FLUX_V']

            # Calculate absolute magnitude if possible
            if distance_pc is not None and apparent_mag is not None:
                distance_pc = float(distance_pc)
                M = apparent_mag - 5 * (np.log10(distance_pc) - 1)
            else:
                M = None

            results.append({
                'Index': idx,
                'RA': coord.ra.deg,
                'Dec': coord.dec.deg,
                'Name': name,
                'Spectral Type': spectral_type,
                'Object Type': otype,
                'Distance (pc)': distance_pc,
                'Apparent Mag (V)': apparent_mag,
                'Absolute Mag (M)': M
            })
        else:
            print(f"No object found for bright spot {idx}")
    except Exception as e:
        print(f"Failed SIMBAD query for object {idx}: {e}")

# ------------------  Make a DataFrame for easy viewing ------------------
df_results = pd.DataFrame(results)

print("\nSIMBAD Results for 10 Brightest Pixels:")
print(df_results)


<ipython-input-88-7bedd5004972>:5: DeprecationWarning: The notation 'flux(V)' is deprecated since 0.4.8 in favor of 'V'. You will see the column appearing with its new name in the output. See section on filters in https://astroquery.readthedocs.io/en/latest/simbad/simbad_evolution.html to see the new ways to interact with SIMBAD's fluxes.
  custom_simbad.add_votable_fields('otype', 'sp', 'distance', 'flux(V)')  # Object type, Spectral Type, Distance, V Magnitude
<ipython-input-88-7bedd5004972>:5: DeprecationWarning: 'distance' has been renamed 'mesdistance'. You will see this new name in the result.
  custom_simbad.add_votable_fields('otype', 'sp', 'distance', 'flux(V)')  # Object type, Spectral Type, Distance, V Magnitude


Failed SIMBAD query for object 0: 'MAIN_ID'
Failed SIMBAD query for object 1: Argument cannot be parsed as a coordinate

SIMBAD Results for 10 Brightest Pixels:
Empty DataFrame
Columns: []
Index: []
